In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR


In [2]:
# 2. Definición de Data Loaders
def get_data_loaders(batch_size=128, pretrained=False):
    transform = transforms.Compose([
        transforms.Resize(224 if pretrained else 32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406] if pretrained else [0.5, 0.5, 0.5],
                             std=[0.229, 0.224, 0.225] if pretrained else [0.5, 0.5, 0.5])
    ])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    return trainloader, testloader

In [3]:
# 3. Definición del modelo AlexNet desde cero
class AlexNetFromScratch(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNetFromScratch, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 4 * 4, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [4]:
# 4. Funciones de entrenamiento y evaluación
def train_model(model, trainloader, criterion, optimizer, scheduler, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print("Starting training...")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader):.4f}")

def evaluate_model(model, testloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Model Accuracy: {100 * correct / total:.2f}%")

In [5]:
# 5. Entrenamiento y evaluación del modelo desde cero
trainloader, testloader = get_data_loaders()
model_scratch = AlexNetFromScratch()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_scratch.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
train_model(model_scratch, trainloader, criterion, optimizer, scheduler, num_epochs=10)
evaluate_model(model_scratch, testloader)


100%|██████████| 170M/170M [00:04<00:00, 41.0MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Starting training...
Epoch 1/10, Loss: 2.2789
Epoch 2/10, Loss: 1.7754
Epoch 3/10, Loss: 1.4759
Epoch 4/10, Loss: 1.2793
Epoch 5/10, Loss: 1.1280
Epoch 6/10, Loss: 0.9684
Epoch 7/10, Loss: 0.8931
Epoch 8/10, Loss: 0.8324
Epoch 9/10, Loss: 0.7692
Epoch 10/10, Loss: 0.7359
Model Accuracy: 73.86%


In [6]:
# 6. Carga y fine-tuning de AlexNet preentrenado
pretrained_model = torchvision.models.alexnet(weights=torchvision.models.AlexNet_Weights.IMAGENET1K_V1)
for param in pretrained_model.features.parameters():
    param.requires_grad = False
pretrained_model.classifier[6] = nn.Linear(4096, 10)
trainloader_pre, testloader_pre = get_data_loaders(pretrained=True)
criterion_pre = nn.CrossEntropyLoss()
optimizer_pre = optim.SGD(pretrained_model.classifier.parameters(), lr=0.01, momentum=0.9)
scheduler_pre = StepLR(optimizer_pre, step_size=5, gamma=0.5)
train_model(pretrained_model, trainloader_pre, criterion_pre, optimizer_pre, scheduler_pre, num_epochs=10)
evaluate_model(pretrained_model, testloader_pre)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 171MB/s]


Files already downloaded and verified
Files already downloaded and verified
Starting training...
Epoch 1/10, Loss: 0.6698
Epoch 2/10, Loss: 0.4829
Epoch 3/10, Loss: 0.4296
Epoch 4/10, Loss: 0.3761
Epoch 5/10, Loss: 0.3415
Epoch 6/10, Loss: 0.2766
Epoch 7/10, Loss: 0.2496
Epoch 8/10, Loss: 0.2318
Epoch 9/10, Loss: 0.2214
Epoch 10/10, Loss: 0.2034
Model Accuracy: 88.45%
